In [1]:
import yfinance as yf
import itertools
from collections import defaultdict

def get_exchange_rates():
    currencies = ['USD', 'EUR', 'JPY', 'GBP', 'CHF', 'CAD', 'AUD', 'NZD', 'CNY', 'INR']
    rates = {}
    for base in currencies:
        for quote in currencies:
            if base != quote:
                pair = f"{base}{quote}=X"
                try:
                    data = yf.Ticker(pair).history(period="1d")
                    if not data.empty:
                        rates[f"{base}/{quote}"] = data['Close'].iloc[-1]
                    else:
                        print(f"No data available for {pair}")
                except Exception as e:
                    print(f"Error fetching data for {pair}: {e}")
    print(rates)
    return rates


/Users/tanishkyadav/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
def calculate_arbitrage(rates, path):
    total_rate = 1
    for i in range(len(path) - 1):
        pair = f"{path[i]}/{path[i+1]}"
        if pair in rates:
            total_rate *= rates[pair]
        else:
            total_rate *= 1 / rates[f"{path[i+1]}/{path[i]}"]
    return total_rate - 1  # Return profit percentage

def find_arbitrage_opportunities(rates):
    currencies = ['USD', 'EUR', 'JPY', 'GBP', 'CHF', 'CAD', 'AUD', 'NZD', 'CNY', 'INR']
    opportunities = []

    for length in range(3, 6):  # 3 to 5 currency combinations
        for path in itertools.permutations(currencies, length):
            path = list(path) + [path[0]]  # Complete the cycle
            profit = calculate_arbitrage(rates, path)
            if profit > 0:
                opportunities.append((path, profit))

    return sorted(opportunities, key=lambda x: x[1], reverse=True)


In [15]:
def main():
    print("Fetching exchange rates...")
    rates = get_exchange_rates()
    
    if not rates:
        print("No exchange rate data available. Exiting.")
        return

    print("Calculating arbitrage opportunities...")
    opportunities = find_arbitrage_opportunities(rates)

    if not opportunities:
        print("No profitable arbitrage opportunities found.")
    else:
        print("\nTop 10 Most Profitable Arbitrage Opportunities:")
        for i, (path, profit) in enumerate(opportunities[:10], 1):
            print(f"{i}. Path: {' -> '.join(path)}")
            print(f"   Potential profit: {profit:.4%}")

    print("\nArbitrage opportunities by number of currencies involved:")
    opportunities_by_length = defaultdict(list)
    for path, profit in opportunities:
        opportunities_by_length[len(path) - 1].append((path, profit))

    for length in range(3, 6):
        top_opportunities = sorted(opportunities_by_length[length], key=lambda x: x[1], reverse=True)[:5]
        print(f"\nTop 5 {length}-currency arbitrage opportunities:")
        for i, (path, profit) in enumerate(top_opportunities, 1):
            print(f"{i}. Path: {' -> '.join(path)}")
            print(f"   Potential profit: {profit:.4%}")

if __name__ == "__main__":
    main()

Fetching exchange rates...
{'USD/EUR': np.float64(0.9332000017166138), 'USD/JPY': np.float64(160.8350067138672), 'USD/GBP': np.float64(0.7907400131225586), 'USD/CHF': np.float64(0.8983200192451477), 'USD/CAD': np.float64(1.3676999807357788), 'USD/AUD': np.float64(1.499400019645691), 'USD/NZD': np.float64(1.6426000595092773), 'USD/CNY': np.float64(7.26669979095459), 'USD/INR': np.float64(83.33499908447266), 'EUR/USD': np.float64(1.0715816020965576), 'EUR/JPY': np.float64(172.38299560546875), 'EUR/GBP': np.float64(0.847350001335144), 'EUR/CHF': np.float64(0.962369978427887), 'EUR/CAD': np.float64(1.4648000001907349), 'EUR/AUD': np.float64(1.6059000492095947), 'EUR/NZD': np.float64(1.7590999603271484), 'EUR/CNY': np.float64(7.7804999351501465), 'EUR/INR': np.float64(89.31900024414062), 'JPY/USD': np.float64(0.00621755188331008), 'JPY/EUR': np.float64(0.0058003999292850494), 'JPY/GBP': np.float64(0.004910999909043312), 'JPY/CHF': np.float64(0.005576999858021736), 'JPY/CAD': np.float64(0.00